In [2]:
#import packages
import os
import numpy as np
import h5py

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential,Model
from keras.layers import Conv2D, MaxPooling2D,Activation, Dropout, Flatten, Dense,Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D  

from keras import applications
from keras import optimizers
from keras.optimizers import SGD
from keras import backend as K

#config the GPU. If you don't use GPU commen this part
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()  
config.gpu_options.allow_growth = True  
config.gpu_options.per_process_gpu_memory_fraction = 0.8
set_session(tf.Session(config=config))
print('configed')

# parameters and data path, save path
img_width, img_height = 224, 224
train_data_dir = 'D:\\EE628\\final_dogs&cats\\data\\train'
validation_data_dir = 'D:\\EE628\\final_dogs&cats\\data\\validation'
predict_data_dir = 'D:\\EE628\\final_dogs&cats\\data\\test'
path_folder='D:\\EE628\\final_dogs&cats\\feature\\'

batch_size = 128

configed


In [2]:
# Set the image Generator
datagen = ImageDataGenerator(rescale=1. / 255)
features_generator_train = datagen.flow_from_directory(train_data_dir,
                                                target_size=(img_width, img_height),
                                                batch_size=batch_size,
                                                class_mode=None,
                                                shuffle=False)
features_generator_validation = datagen.flow_from_directory(validation_data_dir,
                                                target_size=(img_width, img_height),
                                                batch_size=batch_size,
                                                class_mode=None,
                                                shuffle=False)

features_generator_test = datagen.flow_from_directory(predict_data_dir,
                                                target_size=(img_width, img_height),
                                                batch_size=batch_size,
                                                class_mode=None,)

Found 16666 images belonging to 2 classes.
Found 8334 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [4]:
def ExtractFeatures(MODEL,features_generator_train,features_generator_validation,features_generator_test):
    input_tensor = Input(shape=(224, 224, 3))  
    feature_model = MODEL(input_tensor=input_tensor, weights='imagenet', include_top=False)
    gen = ImageDataGenerator()
    features_train = feature_model.predict_generator(features_generator_train)
    features_validation = feature_model.predict_generator(features_generator_validation)
    features_test = feature_model.predict_generator(features_generator_test)
    print(MODEL,'length of train',len(features_train),'length of validation',len(features_validation))
    with h5py.File(path_folder+"Features_%s.h5"%MODEL.__name__) as h:
        h.create_dataset("train", data=features_train)
        h.create_dataset("validation", data=features_validation)
        h.create_dataset("test", data=features_test)
        h.create_dataset("label_train", data=features_generator_train.classes)
        h.create_dataset("label_validation", data=features_generator_validation.classes)
    print(MODEL.__name__,'feature saved')
    

In [3]:
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import VGG16

In [ ]:
#本来想用5个，最后只用了4个，除去Xception是每次用这个都有memory error，还未解决

# Extract features
for MODEL in [ResNet50,InceptionV3,VGG19,VGG16]:
    ExtractFeatures(MODEL,features_generator_train,features_generator_validation,features_generator_test)

In [5]:
ExtractFeatures(VGG16,features_generator_train,features_generator_validation,features_generator_test)

<function VGG16 at 0x0000021B57A199D8> length of train 16666 length of validation 8334
VGG16 feature saved


In [4]:
ExtractFeatures(VGG19,features_generator_train,features_generator_validation,features_generator_test)

<function VGG19 at 0x0000016EFFDB4048> length of train 16666 length of validation 8334
VGG19 feature saved


In [6]:
'''
try to solve memory error problem
import gc


def ExtractFeatures_2(MODEL,features_generator_train,features_generator_validation,features_generator_test):
    input_tensor = Input(shape=(224, 224, 3))  
    feature_model = MODEL(input_tensor=input_tensor, weights='imagenet', include_top=False)
    with h5py.File(path_folder+"Features_%s.h5"%MODEL.__name__) as h:
        features_train = feature_model.predict_generator(features_generator_train)
        h.create_dataset("train", data=features_train)
        del features_train 
        gc.collect()
        
        features_validation = feature_model.predict_generator(features_generator_validation)
        h.create_dataset("validation", data=features_validation)
        del features_validation
        gc.collect()
        
        features_test = feature_model.predict_generator(features_generator_test)
        h.create_dataset("test", data=features_test) 
 
        del features_test
        gc.collect()
        h.create_dataset("label_train", data=features_generator_train.classes)
        h.create_dataset("label_validation", data=features_generator_validation.classes)
    print(MODEL.__name__,'feature saved')
ExtractFeatures_2(Xception,features_generator_train,features_generator_validation,features_generator_test)
'''